In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0].split('BecaNLP')[0],'BecaNLP/Utils'))

import NLPUtils as nlp

%load_ext autoreload
%autoreload 2

In [2]:
from NLPUtils.datasets import imdb

#df = imdb.train_reader().sample(n=100,replace=False,random_state=0)

En cualquier tarea de NLP tengo que definir antes que nada un vocabulario

In [64]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['esto es una prueba',
         'esto es otra prueba',
         'esto es una prueba mas']

vectorizer = CountVectorizer(lowercase=False,max_features=5,ngram_range=(1,2),token_pattern=r'[<>\w]+')
vectorizer.fit(corpus)
print(vectorizer.vocabulary_)
vectorizer.vocabulary_['<UNK>'] = len(vectorizer.vocabulary_)
print(vectorizer.vocabulary_)
print(vectorizer.transform(corpus).toarray())
corpus2 = ['esto es una prueba <UNK>',
         'esto es otra prueba',
         'esto es una prueba mas']
print(vectorizer.transform(corpus2).toarray())

{'esto': 2, 'es': 0, 'prueba': 4, 'esto es': 3, 'es una': 1}
{'esto': 2, 'es': 0, 'prueba': 4, 'esto es': 3, 'es una': 1, '<UNK>': 5}
[[1 1 1 1 1 0]
 [1 0 1 1 1 0]
 [1 1 1 1 1 0]]
[[1 1 1 1 1 1]
 [1 0 1 1 1 0]
 [1 1 1 1 1 0]]


In [66]:
an = vectorizer.build_analyzer()
an('esto es una prueba <UNK> y esto es una prueba también')

['esto',
 'es',
 'una',
 'prueba',
 '<UNK>',
 'y',
 'esto',
 'es',
 'una',
 'prueba',
 'también',
 'esto es',
 'es una',
 'una prueba',
 'prueba <UNK>',
 '<UNK> y',
 'y esto',
 'esto es',
 'es una',
 'una prueba',
 'prueba también']

In [42]:
import re

re.findall(r'\s*[<\w>]+\s*','<esto> es una prueba')

['<esto> ', 'es ', 'una ', 'prueba']

In [149]:
from collections import defaultdict
import array
import numpy as np
import scipy.sparse as sp
import numbers

def _make_int_array():
    """Construct an array.array of a type suitable for scipy.sparse indices."""
    return array.array(str("i"))


class myCountVectorizer(CountVectorizer):
    
    def __init__(self,*args,**kwargs):
        try:
            self.unk_token = kwargs['unk_token']
            kwargs.pop('unk_token')
        except KeyError:
            self.unk_token = None
        super().__init__(*args,**kwargs)
    
    def _count_vocab(self, raw_documents, fixed_vocab, unk_token):
        """Create sparse feature matrix, and vocabulary where fixed_vocab=False
        """
        if fixed_vocab:
            vocabulary = self.vocabulary_
        else:
            # Add a new value when a new vocabulary item is seen
            vocabulary = defaultdict()
            vocabulary.default_factory = vocabulary.__len__

        analyze = self.build_analyzer()
        j_indices = []
        indptr = []

        values = _make_int_array()
        indptr.append(0)
        
        if unk_token is not None:
            for doc in raw_documents:
                feature_counter = {}
                for feature in analyze(doc):
                    try:
                        feature_idx = vocabulary[feature]
                    except KeyError:
                        feature_idx = vocabulary[unk_token]
                        
                    if feature_idx not in feature_counter:
                        feature_counter[feature_idx] = 1
                    else:
                        feature_counter[feature_idx] += 1
                        
                j_indices.extend(feature_counter.keys())
                values.extend(feature_counter.values())
                indptr.append(len(j_indices))
        
        else:
            for doc in raw_documents:
                feature_counter = {}
                for feature in analyze(doc):
                    try:
                        feature_idx = vocabulary[feature]
                        if feature_idx not in feature_counter:
                            feature_counter[feature_idx] = 1
                        else:
                            feature_counter[feature_idx] += 1
                    except KeyError:
                        # Ignore out-of-vocabulary items for fixed_vocab=True
                        continue
    
                j_indices.extend(feature_counter.keys())
                values.extend(feature_counter.values())
                indptr.append(len(j_indices))

        if not fixed_vocab:
            # disable defaultdict behaviour
            vocabulary = dict(vocabulary)
            if not vocabulary:
                raise ValueError("empty vocabulary; perhaps the documents only"
                                 " contain stop words")

        if indptr[-1] > np.iinfo(np.int32).max:  # = 2**31 - 1
            if _IS_32BIT:
                raise ValueError(('sparse CSR array has {} non-zero '
                                  'elements and requires 64 bit indexing, '
                                  'which is unsupported with 32 bit Python.')
                                 .format(indptr[-1]))
            indices_dtype = np.int64

        else:
            indices_dtype = np.int32
        j_indices = np.asarray(j_indices, dtype=indices_dtype)
        indptr = np.asarray(indptr, dtype=indices_dtype)
        values = np.frombuffer(values, dtype=np.intc)

        X = sp.csr_matrix((values, j_indices, indptr),
                          shape=(len(indptr) - 1, len(vocabulary)),
                          dtype=self.dtype)
        X.sort_indices()
        return vocabulary, X
    
    def fit_transform(self, raw_documents, y=None):
        """Learn the vocabulary dictionary and return document-term matrix.
        This is equivalent to fit followed by transform, but more efficiently
        implemented.
        Parameters
        ----------
        raw_documents : iterable
            An iterable which yields either str, unicode or file objects.
        Returns
        -------
        X : array of shape (n_samples, n_features)
            Document-term matrix.
        """
        # We intentionally don't call the transform method to make
        # fit_transform overridable without unwanted side effects in
        # TfidfVectorizer.
        if isinstance(raw_documents, str):
            raise ValueError(
                "Iterable over raw text documents expected, "
                "string object received.")

        self._validate_params()
        self._validate_vocabulary()
        max_df = self.max_df
        min_df = self.min_df
        max_features = self.max_features

        vocabulary, X = self._count_vocab(raw_documents,
                                          self.fixed_vocabulary_,self.unk_token)

        if self.binary:
            X.data.fill(1)

        if not self.fixed_vocabulary_:
            n_doc = X.shape[0]
            max_doc_count = (max_df
                             if isinstance(max_df, numbers.Integral)
                             else max_df * n_doc)
            min_doc_count = (min_df
                             if isinstance(min_df, numbers.Integral)
                             else min_df * n_doc)
            if max_doc_count < min_doc_count:
                raise ValueError(
                    "max_df corresponds to < documents than min_df")
            X, self.stop_words_ = self._limit_features(X, vocabulary,
                                                       max_doc_count,
                                                       min_doc_count,
                                                       max_features)

            X = self._sort_features(X, vocabulary)

            self.vocabulary_ = vocabulary

        return X

    def transform(self, raw_documents):
        """Transform documents to document-term matrix.
        Extract token counts out of raw text documents using the vocabulary
        fitted with fit or the one provided to the constructor.
        Parameters
        ----------
        raw_documents : iterable
            An iterable which yields either str, unicode or file objects.
        Returns
        -------
        X : sparse matrix of shape (n_samples, n_features)
            Document-term matrix.
        """
        if isinstance(raw_documents, str):
            raise ValueError(
                "Iterable over raw text documents expected, "
                "string object received.")
        self._check_vocabulary()

        # use the same matrix-building strategy as fit_transform
        _, X = self._count_vocab(raw_documents, fixed_vocab=True, unk_token=self.unk_token)
        if self.binary:
            X.data.fill(1)
        return X

In [154]:
corpus = ['esto es una prueba',
         'esto es otra prueba',
         'esto es una prueba mas y esto es otra']

vectorizer = myCountVectorizer(lowercase=False,max_features=5,ngram_range=(1,2),
               token_pattern=r'\b\w+\b',unk_token='UNK',vocabulary=['esto','es','UNK'])
vectorizer.fit(corpus)
print(vectorizer.vocabulary_)
X = vectorizer.transform(corpus)
X.toarray()
# vectorizer.vocabulary_['<UNK>'] = len(vectorizer.vocabulary_)
# print(vectorizer.vocabulary_)
# print(vectorizer.transform(corpus).toarray())
# corpus2 = ['esto es una prueba <UNK>',
#          'esto es otra prueba',
#          'esto es una prueba mas']
# print(vectorizer.transform(corpus2).toarray())

{'esto': 0, 'es': 1, 'UNK': 2}


array([[ 1,  1,  5],
       [ 1,  1,  5],
       [ 2,  2, 13]])

In [158]:
vectorizer = CountVectorizer(lowercase=False,max_features=5,ngram_range=(1,2),
               token_pattern=r'\b\w+\b')
vectorizer.fit(corpus)
print(vectorizer.vocabulary_)
X = vectorizer.transform(corpus)
X.toarray()

{'esto': 2, 'es': 0, 'prueba': 4, 'esto es': 3, 'es otra': 1}


array([[1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [2, 1, 2, 2, 1]])

In [67]:
dict([1,2,3,4])

[autoreload of NLPUtils.datasets.utils failed: Traceback (most recent call last):
  File "/home/lestien/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/lestien/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/home/lestien/anaconda3/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/lestien/anaconda3/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/l

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [97]:
from collections import Counter
from itertools import tee, islice
import time

def ngrams(lst, n):
    tlst = lst
    while True:
        a, b = tee(tlst)
        l = tuple(islice(a, n))
        if len(l) == n:
            yield l
            next(b)
            tlst = b
        else:
            break


tokenizer = lambda x: re.findall(r'\b\w+\b',x)
corpus = ['esto es una prueba',
         'esto es otra prueba',
         'esto es una prueba mas'] * 10000

# caso en que vocabulary == None

corpus = imdb.train_reader()['comment']

# tic = time.time()
# tk_to_freq = Counter()
# for text in corpus:
#     tk_to_freq += Counter(tokenizer(text))
# toc = time.time()
# #print(tk_to_freq)
# print(toc-tic)


tic = time.time()
tk_to_freq = Counter(ngrams([tk for text in corpus for tk in tokenizer(text)],2))
toc = time.time()
#print(tk_to_freq)
print(toc-tic)


33.296125173568726


In [98]:
tic = time.time()
tk_to_freq2 = {}
for text in corpus:
    for tk in ngrams(tokenizer(text),2):
        try:
            tk_to_freq2[tk] += 1
        except KeyError:
            tk_to_freq2[tk] = 1
toc = time.time()
#print(tk_to_freq)
print(toc-tic)
print(tk_to_freq2 == tk_to_freq)

38.36709547042847
False


In [106]:
from collections import defaultdict

vocab = defaultdict()
vocab.default_factory = vocab.__len__

vocab[0] = 0
dict(vocab)

{0: 0}

In [221]:
from itertools import chain


corpus = ['esto esto es una prueba',
         'esto es otra prueba',
         'esto es una prueba mas'] * 10000

def ngrams(lst, n):
    if n == 1:
        for i in lst:
            yield i
    else:
        tlst = lst
        for i in range(len(lst)-n+1):
            a, b = tee(tlst)
            yield tuple(islice(a, n))
            next(b)
            tlst = b


vocabulary = defaultdict()
vocabulary.default_factory = vocabulary.__len__
unk_token = 'UNK'
#vocabulary = defaultdict(None,{'esto':0, 'es': 1, 'prueba':2, unk_token:3})
#vocabulary.default_factory = vocabulary.__len__


def tokenizer(doc,vocab):
    tokens = re.findall(r'\b\w+\b',doc)
    return [tk if tk in vocab else unk_token for tk in tokens]

def tokenizer(doc,vocab):
    return re.findall(r'\b\w+\b',doc)


ngram_range = (1,2)

tic = time.time()
for doc in corpus:
    feature_counter = {}
    for i in range(ngram_range[0],ngram_range[1]+1):
        tokens = tokenizer(doc,vocabulary)
        for feature in ngrams(tokens,i):
            try:
                feature_idx = vocabulary[feature]
            except KeyError:
                feature_idx = vocabulary[unk_token]
            if feature_idx not in feature_counter:
                feature_counter[feature_idx] = 1
            else:
                feature_counter[feature_idx] += 1
            
toc = time.time()
print(toc-tic)
print(dict(vocabulary))

1.9278197288513184
{'esto': 0, 'es': 1, 'una': 2, 'prueba': 3, ('esto', 'esto'): 4, ('esto', 'es'): 5, ('es', 'una'): 6, ('una', 'prueba'): 7, 'otra': 8, ('es', 'otra'): 9, ('otra', 'prueba'): 10, 'mas': 11, ('prueba', 'mas'): 12}


In [204]:
a = [1,2,3,4,5,6,7]

def ngrams(lst, n):
    if n == 1:
        for i in lst:
            yield i
    else:
        tlst = lst
        for i in range(len(lst)-n+1):
            a, b = tee(tlst)
            yield tuple(islice(a, n))
            next(b)
            tlst = b

b, c = tee(a)
list(islice(b,3))
list(ngrams(a,1))

[1, 2, 3, 4, 5, 6, 7]

In [ ]:
def ngrams(lst,n):
    

In [ ]:
from collections import defaultdict
import array
import numpy as np
import scipy.sparse as sp
import numbers

def _make_int_array():
    """Construct an array.array of a type suitable for scipy.sparse indices."""
    return array.array(str("i"))


class myCountVectorizer(CountVectorizer):
    
    def __init__(self,token_pattern=r'\b\w+\b',ngram_range=(1,1),min_df=1,max_df=1.0
         max_features=None,vocabulary=None,unk_token=None,dtype=np.int):
        kwargs = {'input':'content',
                  'encoding':'utf-8', 
                  'decode_error':'strict',
                  'strip_accents':None, 
                  'lowercase':False,
                  'preprocessor':None, 
                  'tokenizer':None
                  'stop_words':None,
                  'token_pattern':None,
                  'ngram_range':ngram_range, 
                  'analyzer':self.my_analyzer, 
                  'max_df':max_df, 
                  'min_df':min_df,
                  'max_features':max_features,
                  'vocabulary':vocabulary, 
                  'binary':False, 
                  'dtype':np.int}
        
        self.unk_token = unk_token
        super().__init__(**kwargs)
        
    def my_analyzer(self,token_pattern,vocabulary):
        tokens = re.findall(r'\b\w+\b',doc)
        if vocabulary is None:
            return tokens
        return [tk if tk in vocabulary else self.unk_token for tk in tokens]
        
    
        
    def _count_vocab(self, raw_documents, fixed_vocab, unk_token):
        """Create sparse feature matrix, and vocabulary where fixed_vocab=False
        """
        if fixed_vocab:
            vocabulary = self.vocabulary_
        else:
            # Add a new value when a new vocabulary item is seen
            vocabulary = defaultdict()
            vocabulary.default_factory = vocabulary.__len__

        analyze = self.build_analyzer()
        j_indices = []
        indptr = []

        values = _make_int_array()
        indptr.append(0)
        
        if unk_token is not None:
            for doc in raw_documents:
                feature_counter = {}
                for feature in analyze(doc,vocabulary):
                    try:
                        feature_idx = vocabulary[feature]
                    except KeyError:
                        feature_idx = vocabulary[unk_token]
                        
                    if feature_idx not in feature_counter:
                        feature_counter[feature_idx] = 1
                    else:
                        feature_counter[feature_idx] += 1
                        
                j_indices.extend(feature_counter.keys())
                values.extend(feature_counter.values())
                indptr.append(len(j_indices))
        
        else:
            for doc in raw_documents:
                feature_counter = {}
                for feature in analyze(doc):
                    try:
                        feature_idx = vocabulary[feature]
                        if feature_idx not in feature_counter:
                            feature_counter[feature_idx] = 1
                        else:
                            feature_counter[feature_idx] += 1
                    except KeyError:
                        # Ignore out-of-vocabulary items for fixed_vocab=True
                        continue
    
                j_indices.extend(feature_counter.keys())
                values.extend(feature_counter.values())
                indptr.append(len(j_indices))

        if not fixed_vocab:
            # disable defaultdict behaviour
            vocabulary = dict(vocabulary)
            if not vocabulary:
                raise ValueError("empty vocabulary; perhaps the documents only"
                                 " contain stop words")

        if indptr[-1] > np.iinfo(np.int32).max:  # = 2**31 - 1
            if _IS_32BIT:
                raise ValueError(('sparse CSR array has {} non-zero '
                                  'elements and requires 64 bit indexing, '
                                  'which is unsupported with 32 bit Python.')
                                 .format(indptr[-1]))
            indices_dtype = np.int64

        else:
            indices_dtype = np.int32
        j_indices = np.asarray(j_indices, dtype=indices_dtype)
        indptr = np.asarray(indptr, dtype=indices_dtype)
        values = np.frombuffer(values, dtype=np.intc)

        X = sp.csr_matrix((values, j_indices, indptr),
                          shape=(len(indptr) - 1, len(vocabulary)),
                          dtype=self.dtype)
        X.sort_indices()
        return vocabulary, X
    
    def fit_transform(self, raw_documents, y=None):
        """Learn the vocabulary dictionary and return document-term matrix.
        This is equivalent to fit followed by transform, but more efficiently
        implemented.
        Parameters
        ----------
        raw_documents : iterable
            An iterable which yields either str, unicode or file objects.
        Returns
        -------
        X : array of shape (n_samples, n_features)
            Document-term matrix.
        """
        # We intentionally don't call the transform method to make
        # fit_transform overridable without unwanted side effects in
        # TfidfVectorizer.
        if isinstance(raw_documents, str):
            raise ValueError(
                "Iterable over raw text documents expected, "
                "string object received.")

        self._validate_params()
        self._validate_vocabulary()
        max_df = self.max_df
        min_df = self.min_df
        max_features = self.max_features

        vocabulary, X = self._count_vocab(raw_documents,
                                          self.fixed_vocabulary_,self.unk_token)

        if self.binary:
            X.data.fill(1)

        if not self.fixed_vocabulary_:
            n_doc = X.shape[0]
            max_doc_count = (max_df
                             if isinstance(max_df, numbers.Integral)
                             else max_df * n_doc)
            min_doc_count = (min_df
                             if isinstance(min_df, numbers.Integral)
                             else min_df * n_doc)
            if max_doc_count < min_doc_count:
                raise ValueError(
                    "max_df corresponds to < documents than min_df")
            X, self.stop_words_ = self._limit_features(X, vocabulary,
                                                       max_doc_count,
                                                       min_doc_count,
                                                       max_features)

            X = self._sort_features(X, vocabulary)

            self.vocabulary_ = vocabulary

        return X

    def transform(self, raw_documents):
        """Transform documents to document-term matrix.
        Extract token counts out of raw text documents using the vocabulary
        fitted with fit or the one provided to the constructor.
        Parameters
        ----------
        raw_documents : iterable
            An iterable which yields either str, unicode or file objects.
        Returns
        -------
        X : sparse matrix of shape (n_samples, n_features)
            Document-term matrix.
        """
        if isinstance(raw_documents, str):
            raise ValueError(
                "Iterable over raw text documents expected, "
                "string object received.")
        self._check_vocabulary()

        # use the same matrix-building strategy as fit_transform
        _, X = self._count_vocab(raw_documents, fixed_vocab=True, unk_token=self.unk_token)
        if self.binary:
            X.data.fill(1)
        return X

In [38]:
from collections import defaultdict
import re
from itertools import tee,islice
from NLPUtils.datasets import imdb
import numpy as np
import scipy.sparse as sp

def ngrams(lst, n):
    if n == 1:
        for i in lst:
            yield i
    else:
        tlst = lst
        for i in range(len(lst)-n+1):
            a, b = tee(tlst)
            yield tuple(islice(a, n))
            next(b)
            tlst = b


class myCountVectorizer(object):
    
    def __init__(self,token_pattern=r'\bw+\b',ngram_range=(1,1),
     vocabulary=None,unk_token=None,min_df=1,max_df=1.0,dtype=np.int64):
        
        self.token_pattern = token_pattern
        self.ngram_range = ngram_range
        self.unk_token = unk_token
        self.vocabulary = vocabulary
        self.dtype = dtype
        
    def tokenizer(self,doc,token_pattern,vocabulary,fixed_vocab):
        tokens = re.findall(r'\b\w+\b',doc)
        unk_token = self.unk_token
        if not fixed_vocab:
            return tokens
        return [tk if tk in vocabulary else unk_token for tk in tokens]
        
        
    def _count_vocab(self,corpus,fixed_vocab):
        
        vocab = defaultdict(None,self.vocabulary) if fixed_vocab else defaultdict()
        vocab.default_factory = vocab.__len__
        
        min_ngram, max_ngram = self.ngram_range
        tokenizer = self.tokenizer
        token_pattern = self.token_pattern
        unk_token = self.unk_token
        
        indptr = [0]
        j_indices = []
        values = []
        
        for doc in corpus:
            feature_counter = {}
            for i in range(min_ngram,max_ngram+1):
                tokens = tokenizer(doc,token_pattern,vocab,fixed_vocab)
                for feature in ngrams(tokens,i):
                    try:
                        feature_idx = vocab[feature]
                    except KeyError:
                        feature_idx = vocab[unk_token]
                    if feature_idx not in feature_counter:
                        feature_counter[feature_idx] = 1
                    else:
                        feature_counter[feature_idx] += 1
            print(feature_counter)
            j_indices.extend(feature_counter.keys())
            values.extend(feature_counter.values())
            indptr.append(len(j_indices))
               
        vocab = dict(vocab)
        if unk_token not in vocab and unk_token is not None:
            vocab[unk_token] = len(vocab)
                
        j_indices = np.asarray(j_indices, dtype=self.dtype)
        indptr = np.asarray(indptr, dtype=self.dtype)
        values = np.asarray(values, dtype=self.dtype)
        
        X = sp.csr_matrix((values, j_indices, indptr),
                          shape=(len(indptr) - 1, len(vocab)),
                          dtype=self.dtype)
        X.sort_indices()
        
        true_indices = [vocab[ngram] for ngram in vocab if ngram is not None and (False if isinstance(ngram,tuple) and None in ngram else True)]
        vocab = {tk:idx for idx, tk in enumerate(vocab) if idx in true_indices}
        X = X[:,true_indices]
        
        return X, vocab
        
    def fit(self,corpus):
        
        fixed_vocab = False if self.vocabulary is None else True
        X, vocab = self._count_vocab(corpus,fixed_vocab)
        self.vocabulary = vocab
        
        return X
    
    def transform(self,corpus):
        
        
        

corpus = ['esto esto es una prueba',
         'esto es otra prueba',
         'esto es una prueba mas']

vocabulary = {'esto':0, 'es':1, 'prueba':2}
#vocabulary = None
vectorizer = myCountVectorizer(token_pattern=r'\bw+\b',ngram_range=(1,2),
             vocabulary=vocabulary,unk_token='UNK',min_df=1,max_df=1.0)
vectorizer.fit(corpus)

{0: 2, 1: 1, 4: 1, 2: 1, 5: 1, 6: 1, 7: 1, 8: 1}
{0: 1, 1: 1, 4: 1, 2: 1, 6: 1, 7: 1, 8: 1}
{0: 1, 1: 1, 4: 2, 2: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'esto': 0, 'es': 1, 'prueba': 2, ('una', 'prueba'): 3, 'UNK': 4, ('esto', 'esto'): 5, ('esto', 'es'): 6, ('es', 'UNK'): 7, ('UNK', 'prueba'): 8, ('prueba', 'UNK'): 9}
[[2 1 1 0 1 1 1 1 1 0]
 [1 1 1 0 1 0 1 1 1 0]
 [1 1 1 0 2 0 1 1 1 1]]


In [31]:
from collections import defaultdict
import re
from itertools import tee,islice
from NLPUtils.datasets import imdb
import numpy as np
import scipy.sparse as sp

def ngrams(lst, n):
    if n == 1:
        for i in lst:
            yield i
    else:
        tlst = lst
        for i in range(len(lst)-n+1):
            a, b = tee(tlst)
            yield tuple(islice(a, n))
            next(b)
            tlst = b


class myCountVectorizer(object):
    
    def __init__(self,token_pattern=r'\bw+\b',ngram_range=(1,1),
     vocabulary=None,unk_token=None,min_freq=1,max_freq=np.inf,dtype=np.int64):
        
        self.token_pattern = token_pattern
        self.ngram_range = ngram_range
        self.unk_token = unk_token
        
        if isinstance(vocabulary,list) or isinstance(vocabulary,set) or hasattr(vocabulary,'__iter__'):
            self.vocabulary = {tk: idx for idx, tk in enumerate(vocabulary)}
        elif isinstance(vocabulary,dict) or vocabulary is None:
            self.vocabulary = vocabulary
        else:
            raise TypeError('El vocabulario tiene que ser un iterable de palabras')
            
        self.dtype = dtype
        self.min_freq, self.max_freq = min_freq, max_freq
        
    def tokenizer(self,doc,token_pattern,vocabulary,fixed_vocab,ngram_range):
        tokens = re.findall(r'\b\w+\b',doc)
        unk_token = self.unk_token
        min_ngram, max_ngram = ngram_range
        
        if not fixed_vocab:
            return [tk for i in range(min_ngram,max_ngram+1) for tk in ngrams(tokens,i)]
            
        return [tk if all([sub_tk in vocabulary for sub_tk in tk]) or tk in vocabulary else unk_token for i in range(min_ngram,max_ngram+1) for tk in ngrams(tokens,i)]
        
        
    def _count_vocab(self,corpus,fixed_vocab):
        
        vocab = defaultdict(None,self.vocabulary) if fixed_vocab else defaultdict()
        vocab.default_factory = vocab.__len__
        
        ngram_range = self.ngram_range
        tokenizer = self.tokenizer
        token_pattern = self.token_pattern
        unk_token = self.unk_token
        
        indptr = [0]
        j_indices = []
        values = []
        
        for doc in corpus:
            
            feature_counter = {}
            tokens = tokenizer(doc,token_pattern,vocab,fixed_vocab,ngram_range)
            for feature in tokens:
                try:
                    feature_idx = vocab[feature]
                except KeyError:
                    feature_idx = vocab[unk_token]
                if feature_idx not in feature_counter:
                    feature_counter[feature_idx] = 1
                else:
                    feature_counter[feature_idx] += 1
            j_indices.extend(feature_counter.keys())
            values.extend(feature_counter.values())
            indptr.append(len(j_indices))
               
        vocab = dict(vocab)
        if unk_token not in vocab and unk_token is not None:
            vocab[unk_token] = len(vocab)
                
        j_indices = np.asarray(j_indices, dtype=self.dtype)
        indptr = np.asarray(indptr, dtype=self.dtype)
        values = np.asarray(values, dtype=self.dtype)
        
        X = sp.csr_matrix((values, j_indices, indptr),
                          shape=(len(indptr) - 1, len(vocab)),
                          dtype=self.dtype)
        X.sort_indices()
        
        if self.max_freq < np.inf or self.min_freq > 0:
            sums = np.asarray(X.sum(axis=0)).reshape(-1)
            x1 = sums >= self.min_freq
            x2 = sums <= self.max_freq
            ands = np.logical_and(x1,x2)
            true_indices = list(np.where(ands)[0])
            if unk_token is not None:
                unk_idx = vocab[unk_token]
                if unk_idx not in true_indices:
                    true_indices.append(unk_idx)

        if unk_token is None and unk_token in vocab.keys():
            unk_idx = vocab[unk_token] 
            if unk_idx in true_indices:
                true_indices.remove(unk_idx)
        
        true_indices = sorted(true_indices)
        X = X[:,true_indices]
        tokens = list(vocab.keys())
        vocab = {tokens[i]:idx for idx, i in enumerate(true_indices)}
        
        return X, vocab
        
    def fit(self,corpus):
        
        fixed_vocab = False if self.vocabulary is None else True
        X, vocab = self._count_vocab(corpus,fixed_vocab)
        self.vocabulary = vocab
        
        return X
    
    def transform(self,corpus):
        
        token_pattern = self.token_pattern
        vocab = self.vocabulary
        ngram_range = self.ngram_range
        unk_token = self.unk_token
        
        indptr = [0]
        j_indices = []
        values = []
        
        if unk_token is None:
            for doc in corpus:
                feature_counter = {}
                tokens = self.tokenizer(doc,token_pattern,vocab,True,ngram_range)
                for feature in tokens:
                    try:
                        feature_idx = vocab[feature]
                    except KeyError:
                        continue
                    if feature_idx not in feature_counter:
                        feature_counter[feature_idx] = 1
                    else:
                        feature_counter[feature_idx] += 1
                j_indices.extend(feature_counter.keys())
                values.extend(feature_counter.values())
                indptr.append(len(j_indices))
        else:
            for doc in corpus:
                feature_counter = {}
                tokens = self.tokenizer(doc,token_pattern,vocab,True,ngram_range)
                for feature in tokens:
                    try:
                        feature_idx = vocab[feature]
                    except KeyError:
                        feature_idx = vocab[unk_token]
                    if feature_idx not in feature_counter:
                        feature_counter[feature_idx] = 1
                    else:
                        feature_counter[feature_idx] += 1
                j_indices.extend(feature_counter.keys())
                values.extend(feature_counter.values())
                indptr.append(len(j_indices))
            
        X = sp.csr_matrix((values, j_indices, indptr),
                          shape=(len(indptr) - 1, len(vocab)),
                          dtype=self.dtype)
        X.sort_indices()
        
        return X

corpus_train = ['esto esto es una prueba',
         'esto es otra prueba',
         'esto es una prueba mas']

corpus_dev = ['esto es una prueba para dev',
             'esto también',
             'esto es otra prueba y esto es una prueba más']

#corpus = imdb.train_reader()['comment']
#vocabulary = {'esto':0, 'es':1, ('una','prueba'):2, ('esto','es','una'):3}
vocabulary = None
vectorizer = myCountVectorizer(token_pattern=r'\bw+\b',ngram_range=(1,1),
             vocabulary=vocabulary,unk_token=None,min_freq=1,max_freq=1)

X_train = vectorizer.fit(corpus_train)
X_dev = vectorizer.transform(corpus_dev)
print(vectorizer.vocabulary)
X_dev.toarray()

{'otra': 0, 'mas': 1}


array([[0, 0],
       [0, 0],
       [1, 0]])

La lógica es esta:

* Yo puedo pedir deducir el vocabulario a partir de un corpus. Cuando hago esto, puedo pedir que el programa cuente n-gramas y que se quede con los que tienen una frecuencia dada. En este caso, poner unk_token=None o unk_token='UNK' lo único que cambia es que el token forme parte o no del vocabulario para futuras vectorizaciones. Además, aunque ponga una frecuencia máxima y el unk_token tenga más de esa cuenta, no se elimina del vocabulario. Es decir, se prioriza que haya unk_token a que se elimine esa cantidad, para después trabajar en el dev ser.

* También puedo dar un vocabulario cerrado y pedir que se agregue un unk_token opcional.